In [1]:
from unstructured.partition.auto import partition
from unstructured.chunking.basic import chunk_elements
from unstructured.chunking.title import chunk_by_title
import pandas as pd

In [2]:
text_pdf_filename = "samples/layout-parser-paper.pdf"
table_img_pdf_filename = "samples/embedded-images-tables.pdf"
img_pdf_filename = "samples/embedded-images.pdf"
img_filename = "samples/layout-parser-paper-fast.jpg"

prodesa_alameda_pdf_filename = "samples/prodesa/memoria-alameda-del-rio.pdf"

In [3]:
elements = partition(filename=text_pdf_filename, strategy='hi_res')
# chunks_el = chunk_elements(elements, max_characters=1024, overlap=204)

/home/parjua/.pyenv/versions/3.11.0/envs/unstructured-parser/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [4]:
print("Number of elements:", len(elements))
# print("Number of chunks:", len(chunks_el))

# max([len(chunk.text) for chunk in chunks_el])



Number of elements: 178


In [5]:
image_elememts = partition(filename=text_pdf_filename)

In [6]:
pdf_table_elements = partition(filename=table_img_pdf_filename)

In [42]:
img_pdf_elements = partition(filename=img_pdf_filename)

In [4]:
prodesa_alameda_elements = partition(filename=prodesa_alameda_pdf_filename, strategy='hi_res', pdf_infer_table_structure=True)

The pdf_infer_table_structure kwarg is deprecated. Please use skip_infer_table_types instead.
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
prodesa_alameda_chunks = chunk_by_title(prodesa_alameda_elements, max_characters=1024, overlap=204)

In [6]:
print(prodesa_alameda_chunks[1].text)

1. LOCALIZACIÓN Y VECINOS

El Conjunto Residencial “Pardela” se encuentra en la M-R14 del proyecto de viviendas multiestrato Alameda del Rio, que se construirá en los predios de “El Volador”, ubicado sobre la avenida Circunvalar, entre las carreras 38 y 46, colindando con la nueva sede de la selección Colombia. El plan parcial cuenta con 92 hectáreas destinadas a espacio público entre vías y parques; 89 hectáreas de vivienda, comercio y equipamientos, donde podrán vivir cerca de 22.000 familias bajo un modelo denominado “Ciudad dentro de la Ciudad”. Adicionalmente Alameda del Rio contará con 50 hectáreas de parques, alamedas, andenes y ciclorrutas para recreación y deporte. La Manzana limita al norte con proyecto de vivienda también M-R14, al occidente con proyecto de vivienda también M-R14, al sur con la Carrera 43 y al oriente con una vía vehicular local (1).

La Manzana tiene un área de 14977.53 m2 donde se desarrollará un proyecto de vivienda 4 torres en 15 pisos.


In [9]:
len(prodesa_alameda_chunks)

33

## Parsing

In [37]:
import requests
import json

url = "http://localhost:8081"
headers = {
    "Content-Type": "application/json"
}
data = {
    "flow_id": "LLM_FLW_01HSEZDCTRGPSX11DDWH21K0XX",
    "outputs": ["extraction", "res"],
    "custom_values": {
        "language": "ES_CO",
        # "optional_schema": "node=proyecto;node=ciudad;node=tipologia_proyecto",
        "optional_schema": "N/A",
        "contextual_text": prodesa_alameda_chunks[1].text
    }
}

response = requests.post(url, headers=headers, data=json.dumps(data)).json()["result"]["user_defined_outputs"]
parsed_extraction = response[0]["value"]
parsed_value = response[1]["value"]

print(parsed_extraction)

# Entidades y Relaciones

## Entidades

1. **Conjunto Residencial Pardela**
   - Tipo: Conjunto Residencial
   - Ubicación: M-R14, Proyecto Alameda del Rio
   - Características: 4 torres, 15 pisos

2. **Proyecto Alameda del Rio**
   - Tipo: Proyecto de Vivienda Multiestrato
   - Ubicación: Predios de "El Volador", Avenida Circunvalar, entre carreras 38 y 46
   - Características: 92 hectáreas de espacio público, 89 hectáreas de vivienda, comercio y equipamientos
   - Capacidad: Cerca de 22,000 familias
   - Modelo: "Ciudad dentro de la Ciudad"
   - Infraestructura: 50 hectáreas de parques, alamedas, andenes y ciclorrutas

3. **El Volador**
   - Tipo: Predio
   - Ubicación: Sobre la Avenida Circunvalar

4. **Selección Colombia**
   - Tipo: Sede
   - Ubicación: Colindante con el Proyecto Alameda del Rio

5. **Manzana M-R14**
   - Tipo: Manzana
   - Área: 14,977.53 m²
   - Límites: 
     - Norte: Proyecto de vivienda M-R14
     - Occidente: Proyecto de vivienda M-R14
     - Sur: Carrera 43

In [46]:
nodes = parsed_value["nodes"]
relations = parsed_value["relations"]

nodes_df = pd.DataFrame(nodes)
relations_df = pd.DataFrame(relations)
print(len(nodes_df))
print(len(relations_df))

11
9


In [40]:
nodes_df.head(20)

,type,node_id,props
0,Conjunto Residencial,CONJUNTO_RESIDENCIAL_PARDELA,"[{'name': 'ubicación', 'value': 'M-R14, Proyec..."
1,Proyecto de Vivienda Multiestrato,PROYECTO_ALAMEDA_DEL_RIO,"[{'name': 'ubicación', 'value': 'Predios de ""E..."
2,Predio,EL_VOLADOR,"[{'name': 'ubicación', 'value': 'Sobre la Aven..."
3,Sede,SELECCIÓN_COLOMBIA,"[{'name': 'ubicación', 'value': 'Colindante co..."
4,Manzana,MANZANA_M_R14,"[{'name': 'área', 'value': '14,977.53 m²'}, {'..."
5,Infraestructura,PARQUES_ALAMEDAS_ANDENES_CICLORRUTAS,"[{'name': 'hectáreas', 'value': '50'}]"
6,Capacidad,FAMILIAS,"[{'name': 'cantidad', 'value': 'Cerca de 22,00..."
7,Límite,PROYECTO_VIVIENDA_M_R14_NORTE,"[{'name': 'dirección', 'value': 'Norte'}]"
8,Límite,PROYECTO_VIVIENDA_M_R14_OCCIDENTE,"[{'name': 'dirección', 'value': 'Occidente'}]"
9,Límite,CARRERA_43,"[{'name': 'dirección', 'value': 'Sur'}]"


In [42]:
relations_df.head(20)

,from_node_id,to_node_id,props
0,CONJUNTO_RESIDENCIAL_PARDELA,PROYECTO_ALAMEDA_DEL_RIO,"[{'name': 'tipo', 'value': 'ubicación'}]"
1,PROYECTO_ALAMEDA_DEL_RIO,EL_VOLADOR,"[{'name': 'tipo', 'value': 'ubicación'}]"
2,PROYECTO_ALAMEDA_DEL_RIO,SELECCIÓN_COLOMBIA,"[{'name': 'tipo', 'value': 'colindancia'}]"
3,PROYECTO_ALAMEDA_DEL_RIO,PARQUES_ALAMEDAS_ANDENES_CICLORRUTAS,"[{'name': 'tipo', 'value': 'infraestructura'}]"
4,PROYECTO_ALAMEDA_DEL_RIO,FAMILIAS,"[{'name': 'tipo', 'value': 'capacidad'}]"
5,MANZANA_M_R14,PROYECTO_VIVIENDA_M_R14_NORTE,"[{'name': 'tipo', 'value': 'límite_norte'}]"
6,MANZANA_M_R14,PROYECTO_VIVIENDA_M_R14_OCCIDENTE,"[{'name': 'tipo', 'value': 'límite_occidente'}]"
7,MANZANA_M_R14,CARRERA_43,"[{'name': 'tipo', 'value': 'límite_sur'}]"
8,MANZANA_M_R14,VÍA_VEHICULAR_LOCAL,"[{'name': 'tipo', 'value': 'límite_oriente'}]"


# CSV Loading

In [47]:
def create_csv(nodes_df, relations_df, filename):
    nodes_df.to_csv(filename + "_nodes.csv", index=False)
    relations_df.to_csv(filename + "_relations.csv", index=False)

In [45]:
create_csv(nodes, relations, "outputs/prodesa_alameda")